In [1]:
from watermark_model import Watermark
import torch
import numpy as np
import argparse
from scipy.stats import normaltest
from watermark_model import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_printoptions(sci_mode=False,profile='full')

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='decoder-encoder pretrain')
    parser.add_argument('--secret_length', default=48, type=int)
    args =parser.parse_known_args()[0]
    
model=Watermark(secret_length=args.secret_length).to(device)
model.load_state_dict(torch.load('./model48bit.pth'))
model.eval()

Metric=[]
Loss=[]
for i in range(100):
    secret=torch.Tensor(np.random.choice([0, 1], size=(args.secret_length))).to(device)
    secret = secret.unsqueeze(-1).unsqueeze(-1).unsqueeze(0)
    secret = secret.expand(-1,-1,64,64)
    y, mean, logvar = model(secret)

    mean=mean.reshape(-1,64,64)
    logvar=logvar.reshape(-1,64,64)
    eps = torch.randn_like(logvar)
    std = torch.exp(logvar / 2)
    matrix = eps * std + mean
    average_tensor1 = torch.mean(secret, dim=(-2, -1))
    average_tensor2 = torch.round(torch.mean(y[0], dim=(-2, -1)))
    average_tensor3 = torch.mean(y[0], dim=(-2, -1))
    # print(average_tensor1, average_tensor2, average_tensor3)

    biterror=torch.sum(abs(average_tensor1-average_tensor2))
    mse_loss = torch.nn.MSELoss()
    loss = mse_loss(average_tensor3, average_tensor1)
    Metric.append(biterror)
    Loss.append(loss)
    stat, p1 = normaltest(matrix.view(-1).cpu().detach().clone().numpy())
    # print(p1)

biterror=torch.mean(torch.stack(Metric))
biterrper=biterror/args.secret_length
metric=torch.mean(torch.stack(Loss))
print(f'loss bits',f'{biterror.cpu().detach().numpy()}')
print(f'bit error percentage {biterrper*100:.2f}%')
print(f'loss',f'{metric.cpu().detach()}')

/root/miniconda3/envs/watermark/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 48])) that is different to the input size (torch.Size([48])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss bits 0.07999999821186066
bit error percentage 0.17%
loss 0.0027636217419058084
